In [1]:
import pandas as pd
import numpy as np
import datetime
import gc


In [2]:
## Import up sound alert dependencies
from IPython.display import Audio, display

def allDone():
    display(Audio(url='https://sound.peal.io/ps/audios/000/000/537/original/woo_vu_luvub_dub_dub.wav', autoplay=True))
## Insert whatever audio file you want above

In [3]:
def fix_str_float(ds, col):
    ds[col] = ds[col].str.replace(r'[^0-9\.]','')
    ds[col] = np.where(ds[col]=='',np.nan,ds[col])
    ds[col] = ds[col].astype('float32')
    return ds[col].astype('float32')

In [49]:
# clicks
clicks_df = pd.read_csv('data/clicks.csv', low_memory=False,dtype={'advertiser_id':'int32','action_id':'float32','source_id':'int32','country_code':'category',\
                                                 'latitude':'float32','longitude':'float32','carrier_id':'float32','os_minor':'category',\
                                                  'os_major':'category','specs_brand':'category','timeToClick':'float32','ref_type':'category'\
                                                                  ,'ref_hash':'object'})

clicks_df['touchX'] = fix_str_float(clicks_df,'touchX')
clicks_df['touchY'] = fix_str_float(clicks_df,'touchY')
clicks_df['created'] = pd.to_datetime(clicks_df['created'])
#events
events_df = pd.read_csv('data/events.csv', low_memory=False, dtype={'event_id':'int32','ref_type':'category','application_id':'category',\
                                                                                            'attributed':'bool','device_countrycode':'category','device_city':'category',\
                                                                                            'trans_id':'category','carrier':'category','device_os':'category',\
                                                                                            'connection_type':'category'})
events_df['date'] = pd.to_datetime(events_df['date'])
events_df['wifi'].astype('bool', inplace=True)
events_df.drop(columns=['device_countrycode','session_user_agent','ip_address','device_language'], inplace=True)
# installs
installs_df = pd.read_csv('data/installs.csv', low_memory=False, dtype={'ref_type':'category','application_id':'category',\
                                                      'device_brand':'category','ref_hash':'object','wifi':'category'})
installs_df['kind'] = installs_df['kind'].str.lower()
installs_df['kind'] = installs_df['kind'].astype('category')
installs_df.drop(columns=['session_user_agent','ip_address','device_language','device_model'], inplace=True)
installs_df['created'] = pd.to_datetime(installs_df['created'])
installs_df.drop(['device_countrycode'], axis=1, inplace=True)
# auctions
auctions_df = pd.read_csv('data/auctions.csv', low_memory=False, dtype={'country':'category','platform':'category',\
                                                                        'ref_type_id':'category','source_id':'category','device_id':'object'})

auctions_df['date'] = pd.to_datetime(auctions_df['date'])
allDone()
print('setup done')


setup done


In [39]:
competencia_ids = pd.read_csv('data/target_competencia_ids.csv')
competencia_ids['ref_hash'] = competencia_ids['ref_hash'].str.replace('_sc', '')
competencia_ids['ref_hash'] = competencia_ids['ref_hash'].str.replace('_st', '')
competencia_ids.drop_duplicates(inplace=True)

# Target Auctions

In [40]:
competencia_ids = pd.read_csv('data/target_competencia_ids.csv')
competencia_ids['ref_hash'] = competencia_ids['ref_hash'].str.replace('_sc', '')
competencia_ids['ref_hash'] = competencia_ids['ref_hash'].str.replace('_st', '')
competencia_ids.drop_duplicates(inplace=True)
competencia_ids = competencia_ids['ref_hash']

In [41]:
auctions_sample = auctions_df.loc[auctions_df['device_id'].isin(competencia_ids)].copy()
installs_sample = installs_df.loc[installs_df['ref_hash'].isin(competencia_ids)].copy()
events_sample = events_df.loc[events_df['ref_hash'].isin(competencia_ids)].copy()
del auctions_df
gc.collect()

405

In [42]:
# calculate time in seconds
auctions_sample.drop_duplicates(inplace=True)
auctions_sample = auctions_sample.sort_values(by=['device_id','date'])
#calculate previous time in seconds
auctions_sample['date_prev'] = auctions_sample['date'].shift()
auctions_sample['date_dif_prev'] = auctions_sample['date']- auctions_sample['date_prev']
auctions_sample['device_id_prev'] = auctions_sample['device_id'].astype('object').shift()
auctions_sample['date_dif_prev'] = pd.to_timedelta(np.where(auctions_sample['device_id_prev']==auctions_sample['device_id'], auctions_sample['date_dif_prev'], np.where(auctions_sample['date']-pd.DateOffset(3)<'2019-04-18 00:00:00', auctions_sample['date']-datetime.datetime(2019,4,18), pd.to_timedelta(3, unit='d'))))
auctions_sample['last_seen'] = auctions_sample['date_dif_prev'].dt.total_seconds()
auctions_sample.drop(['device_id_prev','date_dif_prev','date_prev'], axis='columns', inplace=True)
auctions_sample = auctions_sample.sort_values(by=['date'])
#get last date of all devices ids
group_1 = auctions_sample.groupby(['device_id']).agg({'date': 'max'}).reset_index()
auctions_sample = pd.merge(auctions_sample, group_1, on=['device_id', 'date'], how='inner')



In [43]:
#some features
auct_cols = auctions_sample.columns.tolist()
print(1)
#installs and evetns
events_sample['kind'] = 'event'
print(2)
installs_sample['kind'] = installs_sample['kind'].str.replace(' ', '_').str.lower()
print(3)
#information about last installs and events
applications = installs_sample[['ref_hash','created','application_id','kind']].append(events_sample[['ref_hash', 'date', 'application_id','kind']].rename(columns = {'date':'created'}), ignore_index=True)
print(4)
data_1 = pd.merge(auctions_sample, applications, left_on='device_id', right_on='ref_hash', how='left')
print(5)
#only previus installs or events
data_1 = data_1.loc[(data_1['date']>data_1['created']) | data_1['created'].isnull()]
print(6)

# application_id feature by id
app_id_1 = data_1[['application_id','kind']].copy()
print(7)
app_id_1 = pd.get_dummies(app_id_1, dummy_na=True, prefix_sep='=')
print(8)
data_1.drop(columns=['application_id'], inplace=True)
print(9)
data_1 = pd.merge(data_1, app_id_1, left_index=True, right_index=True, how='inner')
print(10)

app_id_1_columns = app_id_1.columns.tolist()
print(11)

group_1 = data_1.groupby(auct_cols).agg({col:'sum' for col in app_id_1_columns})
print(12)
group_1.reset_index(inplace=True)

del installs_sample
del events_sample
gc.collect()
print(13)
auctions_sample = pd.merge(auctions_sample, group_1, on=['date','device_id','ref_type_id','source_id', 'last_seen'], how='left')
print(14)
auctions_sample = auctions_sample.astype({col:'float32' for col in app_id_1_columns})
print(15)

auctions_sample['hora'] = auctions_sample['date'].dt.hour
auctions_sample['dia'] = auctions_sample['date'].dt.day
pivot_auctions = auctions_sample.groupby(['dia','hora']).size().reset_index()
pivot_auctions.columns = ['dia','hora', 'hour_day_dist']
max_auctions = pivot_auctions['hour_day_dist'].max().max()
pivot_auctions['hour_day_dist'] = pivot_auctions['hour_day_dist']/max_auctions
display(pivot_auctions.head(5))
print(auctions_sample.shape)
auctions_sample = pd.merge(auctions_sample, pivot_auctions, on=['dia','hora'], how='left')
auctions_sample.drop(columns=['hora','dia'], inplace=True)
print(auctions_sample.shape)

auctions_sample.fillna(value={'application_id=nan':1,'kind=nan':1}, inplace=True)
print(16)
auctions_sample.fillna(value={col:0 for col in app_id_1_columns}, inplace=True)
print(17)
auctions_sample = auctions_sample.astype({col:'int32' for col in app_id_1_columns})
print(18)
auctions_sample.reset_index(inplace=True, drop=True)
print(19)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


,dia,hora,hour_day_dist
0,18,0,0.004796
1,18,1,0.011990
2,18,2,0.004796
3,18,3,0.011990
4,18,4,0.007194


(4037, 182)
(4037, 181)
16
17
18
19


In [46]:
#cols from auctions sample
auctions_cols = ['application_id=101', 'application_id=116', 'application_id=117', 'application_id=121', 'application_id=122', 'application_id=123', 'application_id=126', 'application_id=128', 'application_id=129', 'application_id=13', 'application_id=133', 'application_id=135', 'application_id=14', 'application_id=140', 'application_id=143', 'application_id=145', 'application_id=147', 'application_id=153', 'application_id=155', 'application_id=158', 'application_id=159', 'application_id=161', 'application_id=163', 'application_id=165', 'application_id=167', 'application_id=170', 'application_id=175', 'application_id=180', 'application_id=185', 'application_id=187', 'application_id=188', 'application_id=19', 'application_id=194', 'application_id=196', 'application_id=197', 'application_id=200', 'application_id=203', 'application_id=21', 'application_id=210', 'application_id=212', 'application_id=214', 'application_id=218', 'application_id=226', 'application_id=230', 'application_id=232', 'application_id=256', 'application_id=26', 'application_id=261', 'application_id=263', 'application_id=265', 'application_id=268', 'application_id=269', 'application_id=27', 'application_id=274', 'application_id=279', 'application_id=289', 'application_id=291', 'application_id=3', 'application_id=30', 'application_id=301', 'application_id=302', 'application_id=305', 'application_id=308', 'application_id=309', 'application_id=31', 'application_id=311', 'application_id=327', 'application_id=330', 'application_id=339', 'application_id=34', 'application_id=35', 'application_id=36', 'application_id=39', 'application_id=4', 'application_id=42', 'application_id=45', 'application_id=49', 'application_id=54', 'application_id=55', 'application_id=56', 'application_id=57', 'application_id=62', 'application_id=65', 'application_id=66', 'application_id=68', 'application_id=70', 'application_id=71', 'application_id=72', 'application_id=75', 'application_id=77', 'application_id=78', 'application_id=80', 'application_id=81', 'application_id=86', 'application_id=87', 'application_id=94', 'application_id=96', 'application_id=nan', 'convertion=install', 'convertion=nan', 'date', 'device_id', 'hour_day_dist', 'last_seen', 'ref_type_id', 'source_id']


In [47]:
for col in auctions_cols:
    if not(col in auctions_sample.columns):
        auctions_sample[col] = 0
auctions_sample = auctions_sample[auctions_cols]

In [44]:
auctions_sample.shape

(4037, 181)

In [45]:
competencia_ids.shape

(4037,)

In [51]:
#export to file
auctions_sample.to_csv('data/.csv', index=False) 

In [52]:
del auctions_sample
gc.collect()

144

# Target Convertions

In [69]:
installs_sample = installs_df.loc[installs_df['ref_hash'].isin(competencia_ids)].copy()
events_sample = events_df.loc[events_df['ref_hash'].isin(competencia_ids)].copy()

In [70]:
#calculate previous time in seconds
installs_sample = installs_sample.sort_values(by=['ref_hash','created'])
installs_sample['date_prev'] = installs_sample['created'].shift()
installs_sample['date_dif_prev'] = installs_sample['created']- installs_sample['date_prev']
installs_sample['device_id_prev'] = installs_sample['ref_hash'].astype('object').shift()
installs_sample['date_dif_prev'] = pd.to_timedelta(np.where(installs_sample['device_id_prev']==installs_sample['ref_hash'], installs_sample['date_dif_prev'], np.where(installs_sample['created']-pd.DateOffset(3)<'2019-04-18 00:00:00', installs_sample['created']-datetime.datetime(2019,4,18), pd.to_timedelta(3, unit='d'))))
installs_sample['last_seen'] = installs_sample['date_dif_prev'].dt.total_seconds()
installs_sample.drop(['device_id_prev','date_dif_prev','date_prev'], axis='columns', inplace=True)
installs_sample = installs_sample.sort_values(by=['created'])

#average time
installs_sample['hora'] = installs_sample['created'].dt.hour
installs_sample['dia'] = installs_sample['created'].dt.day
pivot_auctions = installs_sample.groupby(['dia','hora']).size().reset_index()
pivot_auctions.columns = ['dia','hora', 'hour_day_dist']
max_auctions = pivot_auctions['hour_day_dist'].max().max()
pivot_auctions['hour_day_dist'] = pivot_auctions['hour_day_dist']/max_auctions
display(pivot_auctions.head(5))
installs_sample = pd.merge(installs_sample, pivot_auctions, on=['dia','hora'], how='left')
installs_sample.drop(columns=['hora','dia'], inplace=True)




,dia,hora,hour_day_dist
0,18,0,0.514706
1,18,1,0.529412
2,18,2,0.823529
3,18,3,0.720588
4,18,4,0.397059


In [71]:
#calculate previous time in seconds
events_sample = events_sample.rename(columns = {'date':'created'}).sort_values(by=['ref_hash','created'])
events_sample['date_prev'] = events_sample['created'].shift()
events_sample['date_dif_prev'] = events_sample['created']- events_sample['date_prev']
events_sample['device_id_prev'] = events_sample['ref_hash'].astype('object').shift()
events_sample['date_dif_prev'] = pd.to_timedelta(np.where(events_sample['device_id_prev']==events_sample['ref_hash'], events_sample['date_dif_prev'], np.where(events_sample['created']-pd.DateOffset(3)<'2019-04-18 00:00:00', events_sample['created']-datetime.datetime(2019,4,18), pd.to_timedelta(3, unit='d'))))
events_sample['last_seen'] = events_sample['date_dif_prev'].dt.total_seconds()
events_sample.drop(['device_id_prev','date_dif_prev','date_prev'], axis='columns', inplace=True)
events_sample = events_sample.sort_values(by=['created'])
events_sample['user_agent']=np.nan
events_sample['kind']='event' #it is hashed

#average time
events_sample['hora'] = events_sample['created'].dt.hour
events_sample['dia'] = events_sample['created'].dt.day
pivot_auctions = events_sample.groupby(['dia','hora']).size().reset_index()
pivot_auctions.columns = ['dia','hora', 'hour_day_dist']
max_auctions = pivot_auctions['hour_day_dist'].max().max()
pivot_auctions['hour_day_dist'] = pivot_auctions['hour_day_dist']/max_auctions
display(pivot_auctions.head(5))
events_sample = pd.merge(events_sample, pivot_auctions, on=['dia','hora'], how='left')
events_sample.drop(columns=['hora','dia'], inplace=True)



,dia,hora,hour_day_dist
0,18,0,0.332133
1,18,1,0.377138
2,18,2,0.504050
3,18,3,0.383438
4,18,4,0.372637


In [135]:
#some features
install_cols = installs_sample.columns.tolist()
print(1)
#information about last installs and events
convertions = installs_sample[['ref_hash','created','application_id','user_agent','wifi','kind','last_seen','hour_day_dist']].append(events_sample[['ref_hash', 'created', 'application_id','user_agent','wifi','kind','last_seen','hour_day_dist']], ignore_index=True)

print(2)

#user agent
convertions['user_agent'] = np.where(convertions['user_agent'].str.contains('Android', regex=False),'Android',convertions['user_agent'])
convertions['user_agent'] = np.where(convertions['user_agent'].str.contains('Darwin', regex=False) | convertions['user_agent'].str.contains('iOS', regex=False),'iOS',convertions['user_agent'])
print(3)

# previus applications

app_id_1 = convertions[['application_id','created','ref_hash']].copy()
group_1 = pd.merge(convertions, app_id_1, on='ref_hash', how='inner')
group_1 = group_1.loc[(group_1['created_x']>group_1['created_y']) | group_1['created_y'].isnull()]
group_1['application_id'] = group_1['application_id_y']
group_1['created'] = group_1['created_x']
group_1.drop(columns=['application_id_y','created_x', 'application_id_x'], inplace=True)
group_1 = group_1.loc[group_1['created_y']>group_1['created']-pd.DateOffset(7)]
group_1 = pd.get_dummies(group_1,prefix_sep='=', dummy_na=True,columns=['application_id'],drop_first=True)
app_id_1_columns = []
for col in group_1.columns.tolist():
    if col.startswith('application_id'):
        app_id_1_columns.append(col)
group_1 = group_1.groupby(['created','ref_hash']).agg({col:'sum' for col in app_id_1_columns})
convertions = pd.merge(convertions, group_1, on=['created','ref_hash'], how='left')
convertions['curr_app_id'] = convertions['application_id']
convertions.drop(columns=['application_id'], inplace=True)
print(4)



convertions = pd.get_dummies(convertions,prefix_sep='=', dummy_na=True,columns=['curr_app_id','wifi', 'kind', 'user_agent'],drop_first=True)

convertions.fillna(value={'application_id=nan':1,'kind=nan':1,'curr_app_id=nan':1}, inplace=True)
print(6)
convertions.fillna(value={col:0 for col in app_id_1_columns}, inplace=True)
print(7)
convertions = convertions.astype({col:'int32' for col in app_id_1_columns})
print(8)
convertions.reset_index(inplace=True, drop=True)
print(9)


1
2
3
4
6
7
8
9


In [140]:
print(convertions.shape)

# only last info
convertions['ref_hash'] = convertions['ref_hash'].map(str)
group_1 = convertions.groupby(['ref_hash']).agg({'created': 'max'}).reset_index()
print(group_1.shape)
convertions2 = pd.merge(convertions, group_1, on=['ref_hash','created'], how='inner')
print(convertions2.shape)

(83967, 315)
(4037, 2)
(4161, 315)


In [143]:
convertions2.loc[convertions2['ref_hash']=='859094624921827223']

,ref_hash,created,last_seen,hour_day_dist,application_id=106,application_id=109,application_id=112,application_id=116,application_id=117,application_id=121,...,kind=reinstall,kind=sessionbegin,kind=sign in,kind=sign up,kind=signed in,kind=startsessionplayback,kind=terms_open_split,kind=nan,user_agent=iOS,user_agent=nan
555,859094624921827223,2019-04-23 02:17:01.880,0.073,0.514706,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
556,859094624921827223,2019-04-23 02:17:01.880,259200.000,0.505851,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [136]:
convertions.dtypes

ref_hash                                    object
created                             datetime64[ns]
last_seen                                  float64
hour_day_dist                              float64
application_id=106                           int32
application_id=109                           int32
application_id=112                           int32
application_id=116                           int32
application_id=117                           int32
application_id=121                           int32
application_id=122                           int32
application_id=126                           int32
application_id=128                           int32
application_id=13                            int32
application_id=133                           int32
application_id=135                           int32
application_id=14                            int32
application_id=140                           int32
application_id=143                           int32
application_id=145             

In [129]:
group_1.loc[group_1['ref_hash'].isnull()]

,ref_hash,created
0,NaN,2019-04-26 21:32:45.082
1,NaN,2019-04-21 14:56:50.145
2,NaN,2019-04-26 23:17:25.394
3,NaN,2019-04-26 23:31:12.335
4,NaN,2019-04-22 04:28:59.644
5,NaN,2019-04-26 00:54:12.097
6,NaN,2019-04-25 17:17:57.460
7,NaN,2019-04-25 01:22:39.744
8,NaN,2019-04-26 08:38:26.645
9,NaN,2019-04-23 04:28:05.374


In [121]:
convertions.loc[convertions['ref_hash'].isnull()]

,ref_hash,created,last_seen,hour_day_dist,application_id=106,application_id=109,application_id=112,application_id=116,application_id=117,application_id=121,...,kind=reinstall,kind=sessionbegin,kind=sign in,kind=sign up,kind=signed in,kind=startsessionplayback,kind=terms_open_split,kind=nan,user_agent=iOS,user_agent=nan
